In [1]:
# Check GPU
!nvidia-smi

Sat Oct  9 05:23:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
import cv2
import torch

In [4]:
train_annots = pd.read_csv('/content/drive/MyDrive/object_detection/data/train_annots_v2.csv', header=None)
train_annots.head(2)

,0,1,2,3,4,5
0,/content/drive/MyDrive/object_detection/data/t...,846,145,992,622,person
1,/content/drive/MyDrive/object_detection/data/t...,848,216,1023,767,person


# Training the Object Detection Model

In [5]:
os.chdir('/content/drive/MyDrive/object_detection/src/pytorch-retinanet')

In [ ]:
# import time
# start = time.time()
# !python train.py \
# --dataset csv \
# --csv_train  '/content/drive/MyDrive/object_detection/data/train_annots_v2.csv' \
# --csv_classes '/content/drive/MyDrive/object_detection/data/class_label.csv'  \
# --csv_val '/content/drive/MyDrive/object_detection/data/val_annots_v2.csv' \
# --depth 18 \
# --epochs 10
# end = time.time()
# print("Total time taken in sec ", np.round((end - start)/60, 3))

Streaming output truncated to the last 5000 lines.
Epoch: 0 | Iteration: 504 | Classification loss: 0.50369 | Regression loss: 0.94188 | Running loss: 1.43745
Epoch: 0 | Iteration: 505 | Classification loss: 0.34098 | Regression loss: 0.68063 | Running loss: 1.43533
Epoch: 0 | Iteration: 506 | Classification loss: 0.52925 | Regression loss: 0.92148 | Running loss: 1.43419
Epoch: 0 | Iteration: 507 | Classification loss: 0.48733 | Regression loss: 0.89721 | Running loss: 1.43282
Epoch: 0 | Iteration: 508 | Classification loss: 0.48605 | Regression loss: 0.80761 | Running loss: 1.43145
Epoch: 0 | Iteration: 509 | Classification loss: 0.49509 | Regression loss: 0.76864 | Running loss: 1.43001
Epoch: 0 | Iteration: 510 | Classification loss: 0.52206 | Regression loss: 0.84095 | Running loss: 1.42900
Epoch: 0 | Iteration: 511 | Classification loss: 0.55601 | Regression loss: 0.88282 | Running loss: 1.42822
Epoch: 0 | Iteration: 512 | Classification loss: 0.45612 | Regression loss: 0.81115 |

# Final Metrics after 5 Epochs

- Pretrained Weights Used -- Resnet Coco
- **IOU Threshold used :- 0.5**
- **Score Threshold for Box removal :- 0.5**
- **mAP (Person): 0.421**
- **mAP (Car): 0.514**


- ***mAP (Complete System) : 0.468***

# Visualizing the Predictions from the trained RetinaNet Model

In [3]:
# !python visualize_single_image.py \
# --image_dir '/content/drive/MyDrive/object_detection/data/trainval/images/' \
# --class_list '/content/drive/MyDrive/object_detection/data/class_label.csv'  \
# --model_path '/content/drive/MyDrive/object_detection/src/pytorch-retinanet/csv_retinanet_5.pt'
# # --csv_val '/content/drive/MyDrive/object_detection/data/val_annots_v2.csv' \


In [27]:
import torch
import numpy as np
import time
import os
import csv
import cv2
import argparse


def load_classes(csv_reader):
    result = {}

    for line, row in enumerate(csv_reader):
        line += 1

        try:
            class_name, class_id = row
        except ValueError:
            raise(ValueError('line {}: format should be \'class_name,class_id\''.format(line)))
        class_id = int(class_id)

        if class_name in result:
            raise ValueError('line {}: duplicate class name: \'{}\''.format(line, class_name))
        result[class_name] = class_id
    return result


# Draws a caption above the box in an image
def draw_caption(image, box, caption):
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)


def detect_image(image_path, model_path, class_list):

    with open(class_list, 'r') as f:
        classes = load_classes(csv.reader(f, delimiter=','))

    labels = {}
    for key, value in classes.items():
        labels[value] = key

    model = torch.load(model_path)

    if torch.cuda.is_available():
        model = model.cuda()

    model.training = False
    model.eval()

    for img_name in os.listdir(image_path):

        image = cv2.imread(os.path.join(image_path, img_name))
        if image is None:
            continue
        image_orig = image.copy()

        rows, cols, cns = image.shape

        smallest_side = min(rows, cols)

        # rescale the image so the smallest side is min_side
        min_side = 608
        max_side = 1024
        scale = min_side / smallest_side

        # check if the largest side is now greater than max_side, which can happen
        # when images have a large aspect ratio
        largest_side = max(rows, cols)

        if largest_side * scale > max_side:
            scale = max_side / largest_side

        # resize the image with the computed scale
        image = cv2.resize(image, (int(round(cols * scale)), int(round((rows * scale)))))
        rows, cols, cns = image.shape

        pad_w = 32 - rows % 32
        pad_h = 32 - cols % 32

        new_image = np.zeros((rows + pad_w, cols + pad_h, cns)).astype(np.float32)
        new_image[:rows, :cols, :] = image.astype(np.float32)
        image = new_image.astype(np.float32)
        image /= 255
        image -= [0.485, 0.456, 0.406]
        image /= [0.229, 0.224, 0.225]
        image = np.expand_dims(image, 0)
        image = np.transpose(image, (0, 3, 1, 2))

        with torch.no_grad():

            image = torch.from_numpy(image)
            if torch.cuda.is_available():
                image = image.cuda()

            st = time.time()
            print(image.shape, image_orig.shape, scale)
            scores, classification, transformed_anchors = model(image.cuda().float())
            print('Elapsed time: {}'.format(time.time() - st))
            idxs = np.where(scores.cpu() > 0.5)

            for j in range(idxs[0].shape[0]):
                bbox = transformed_anchors[idxs[0][j], :]

                x1 = int(bbox[0] / scale)
                y1 = int(bbox[1] / scale)
                x2 = int(bbox[2] / scale)
                y2 = int(bbox[3] / scale)
                label_name = labels[int(classification[idxs[0][j]])]
                print(bbox, classification.shape)
                score = scores[j]
                caption = '{} {:.3f}'.format(label_name, score)
                # draw_caption(img, (x1, y1, x2, y2), label_name)
                draw_caption(image_orig, (x1, y1, x2, y2), caption)
                cv2.rectangle(image_orig, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)

            cv2_imshow(image_orig)
            # cv2.waitKey(0)

In [28]:
from google.colab.patches import cv2_imshow
image_dir = '/content/drive/MyDrive/object_detection/data/trainval/images/'
class_list = '/content/drive/MyDrive/object_detection/data/class_label.csv'
model_path = '/content/drive/MyDrive/object_detection/src/pytorch-retinanet/csv_retinanet_5.pt'

In [ ]:
val_annots = pd.read_csv('/content/drive/MyDrive/object_detection/data/val_annots_v2.csv', header=None)
val_annots.head(2)

In [4]:
detect_image(image_dir, model_path, class_list)

NameError: name 'detect_image' is not defined